In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# This Python 3 environment comes with many helpfhttp://localhost:8890/notebooks/m-3/Classification-Final-release-repeated.ipynb#ul analytics libraries installed
import numpy as np
import pandas as pd
from datetime import datetime

import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error

from tensorflow import keras
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense,LSTM,GRU,Dropout,Masking,TimeDistributed,RepeatVector,GRU,SimpleRNN,BatchNormalization
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import tensorflow as tf


import math
import os
import datetime

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
gr = "alive"
dbname = "MIMIC-III"
hr_data = 48
hr_lab = int(48/8)

In [ ]:
col_vital = ['diastolicbp', 'systolicbp',
       'heartrate', 'meanbp', 'temperature', 'spo2', 'respiration',]

col_lab = [
       'bicarbonate', 'bun', 'chloride', 'creatinine', 'glucose', 'hemoglobin',
       'hematocrit', 'lactate', 'platelet', 'potassium', 'sodium', 'wbc',
       'bilirubin', 'albumin', 'bands', 'ptt']

## Load data

In [ ]:
db_vital = pd.read_pickle('db_vital48-no-scale.pickle')
db_lab = pd.read_pickle('db_lab48-no-scale.pickle')

## Split dataset

In [ ]:
def oversample(X, Y):
    num_pos = np.sum(Y==1)
    num_neg = np.sum(Y==0)
    #print(f'#Pos: {num_pos} #Neg: {num_neg} >> ', end='')

    idxpos = np.where(Y==1)[0]
    idxneg = np.where(Y==0)[0]
    
    rng = np.random.default_rng(1)
    choices = rng.choice(idxpos, num_neg)
    #print(f'#Pos: {len(choices)} #Neg: {num_neg}')

    RX = np.concatenate([X[choices], X[idxneg]], axis=0)
    RY = np.concatenate([Y[choices], Y[idxneg]], axis=0)
    
    return RX, RY

def undersample(X, Y):
    num_pos = np.sum(Y==1)
    num_neg = np.sum(Y==0)
    #print(f'#Pos: {num_pos} #Neg: {num_neg} >> ', end='')

    idxpos = np.where(Y==1)[0]
    idxneg = np.where(Y==0)[0]

    rng = np.random.default_rng(1)
    choices = rng.choice(idxneg, num_pos)
    #print(f'#Pos: {num_pos} #Neg: {len(choices)}')

    RX = np.concatenate([X[idxpos], X[choices]], axis=0)
    RY = np.concatenate([Y[idxpos], Y[choices]], axis=0)
    
    return RX, RY

In [ ]:
def data_prep(df, cols, hrs, ktrainid, kvalid, ktestid, mode='oversample', baseline=False):
    values = df.values

    values_train = values[ktrainid]
    values_validation = values[kvalid]
    values_test = values[ktestid]

    trainX, trainY = values_train[:,:-2], values_train[:,-2]
    valX, valY = values_validation[:,:-2], values_validation[:,-2]
    testX, testY = values_test[:,:-2], values_test[:,-2]

    if baseline == False:
        trainX = trainX.reshape(trainX.shape[0],hrs,len(cols))
        valX = valX.reshape(valX.shape[0],hrs,len(cols))
        testX = testX.reshape(testX.shape[0],hrs,len(cols))
    
    #print(f'TrainX: {str(trainX.shape)} TrainY: {str(trainY.shape)}  [{np.sum(trainY==0)}+{np.sum(trainY==1)}]')
    #print(f'ValidX: {str(valX.shape)}  ValidX: {str(valY.shape)}   [{np.sum(valY==0)}+{np.sum(valY==1)}]')
    #print(f'TestX:  {str(testX.shape)}  TestX:  {str(testY.shape)}   [{np.sum(testY==0)}+{np.sum(testY==1)}]')


    if mode == 'oversample':
        trainX, trainY = oversample(trainX, trainY)
        valX, valY = oversample(valX, valY)
        testX, testY = oversample(testX, testY)
    elif mode == 'undersample':
        trainX, trainY = undersample(trainX, trainY)
        valX, valY = undersample(valX, valY)
        testX, testY = undersample(testX, testY)
    
    return (trainX, trainY), (valX, valY), (testX, testY)

## Custom Model Callback

In [ ]:
class CustomLearningRateScheduler(keras.callbacks.Callback):
    def __init__(self, schedule):
        super(CustomLearningRateScheduler, self).__init__()
        self.schedule = schedule

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        # Get the current learning rate from model's optimizer.
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        # Call schedule function to get the scheduled learning rate.
        scheduled_lr = self.schedule(epoch, lr)
        # Set the value back to the optimizer before this epoch starts
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        #print("\nEpoch %05d: Learning rate is %f." % (epoch, scheduled_lr))
    

def lr_schedule(epoch, lr):
    """Helper function to retrieve the scheduled learning rate based on epoch."""
    m = 1
    if( epoch % m == m or epoch % m == 0):
        drop = 0.055
        epochs_drop = 2.0
        lrate = lr * 1/(1 + drop * epoch)
    else:
        lrate = lr

    return lrate

class LossAndErrorPrintingCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print()
        yhat = model.predict(testX, verbose=0)
        diff = value_restore(yhat, train_mean, train_std) -  value_restore(testy, train_mean, train_std)
        for i, col in enumerate(column_indices):
            print("{} > mae = {:f}".format(col, np.abs(diff[:,:,i]).mean()))

## Models

In [ ]:
def baseline_model(trainXY, valXY, testXY, hrs, k, arch, name, verbose=1):

    print(f'MODEL {name.upper()}', end='\t')
    
    with tf.device('/gpu:0'):
        model = Sequential()
        model.add(Dense(input_shape=[trainXY[0].shape[1]],activation = 'relu', units=64))
        model.add(BatchNormalization())
        model.add(Dense(units=64))
        model.add(BatchNormalization())
        model.add(Dense(units=64))
        model.add(BatchNormalization())
        model.add(Dense(units=1, activation='sigmoid'))
        
        
    architecture = '{}-vital-{}-k{}'.format(arch, hrs, k)
    logdir = os.path.join("logs-class","{}-{}".format(architecture,
                                                      datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))

    tensorboard_callback = [tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
                            ,CustomLearningRateScheduler(lr_schedule)
                            ,EarlyStopping(monitor='val_loss', patience=10)
                            ,ModelCheckpoint(filepath="./{}/{}-best-{}-hr.h5".format(logdir,architecture,hrs)
                            ,monitor='val_loss'
                            ,save_best_only=True)]
    model.compile(loss='binary_crossentropy',
                  optimizer=tf.optimizers.Adam(0.01, beta_1=0.1, beta_2=0.001, amsgrad=True),
                  metrics=['accuracy'])
    
    with tf.device('/gpu:0'):
        model.fit(trainXY[0], trainXY[1], 
                        validation_data=valXY,
                        shuffle=True, epochs=20, verbose=verbose,
                        callbacks=[tensorboard_callback])

        validYpred = model.predict(valXY[0])
        testYpred = model.predict(testXY[0])
        
        # OPTIMAL THRESHOLD
        fpr, tpr, thresholds = sklearn.metrics.roc_curve(valXY[1], validYpred, pos_label=1)
        optimal_idx = np.argmax(tpr - fpr)
        optimal_threshold = thresholds[optimal_idx]

        # PERFORMANCE
        fpr, tpr, thresholds = sklearn.metrics.roc_curve(testXY[1], testYpred, pos_label=1)
        auc = sklearn.metrics.auc(fpr, tpr)
        
        tn, fp, fn, tp  = sklearn.metrics.confusion_matrix(testXY[1], testYpred>optimal_threshold).ravel()
        macc = (tp+tn)/(tp+tn+fp+fn)
        msen = tp/(tp+fn)
        mspc = tn/(tn+fp)
        print(f'AUC: {auc:.4f}\tAccuracy: {macc:.4f}\tSensitivity: {msen:.4f}\tSpecificity {mspc:.4f}')
    
    return model, (testXY[1], testYpred)

In [ ]:
def rnn_model(trainXY, valXY, testXY, hrs, k, arch, name, verbose=1):
    print(f'MODEL {name.upper()}', end='\t')
    
    with tf.device('/gpu:0'):
        model = Sequential()
        model.add(SimpleRNN(input_shape=(trainXY[0].shape[1], trainXY[0].shape[2]), units=16,
                       dropout=0.0, recurrent_dropout=0.0,
                       return_sequences=True, return_state=False,
                       stateful=False, unroll=False
                      ))
        model.add(BatchNormalization())
        model.add(SimpleRNN(units=16,
                       dropout=0.0, recurrent_dropout=0.0,
                       return_sequences=True, return_state=False,
                       stateful=False, unroll=False
                      ))
        model.add(BatchNormalization())
        model.add(SimpleRNN(units=16,
                       dropout=0.0, recurrent_dropout=0.0,
                       return_sequences=False, return_state=False,
                       stateful=False, unroll=False
                      ))
        model.add(BatchNormalization())
        model.add(Dense(units=1, activation='sigmoid'))


    architecture = '{}-vital-{}-k{}'.format(arch, hrs, k)
    logdir = os.path.join("logs-class","{}-{}".format(architecture,
                                                      datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))

    tensorboard_callback = [tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
                            ,CustomLearningRateScheduler(lr_schedule)
                            ,EarlyStopping(monitor='val_loss', patience=10)
                            ,ModelCheckpoint(filepath="./{}/{}-best-{}-hr.h5".format(logdir,architecture,hrs)
                            ,monitor='val_loss'
                            ,save_best_only=True)]
    model.compile(loss='binary_crossentropy',
                  optimizer=tf.optimizers.Adam(0.005, beta_1=0.1, beta_2=0.001, amsgrad=True),
                  metrics=['accuracy'])
    
    with tf.device('/gpu:0'):
        model.fit(trainXY[0], trainXY[1], 
                        validation_data=valXY,
                        shuffle=True, epochs=20, verbose=verbose,
                        callbacks=[tensorboard_callback])

        validYpred = model.predict(valXY[0])
        testYpred = model.predict(testXY[0])
        
        # OPTIMAL THRESHOLD
        fpr, tpr, thresholds = sklearn.metrics.roc_curve(valXY[1], validYpred, pos_label=1)
        optimal_idx = np.argmax(tpr - fpr)
        optimal_threshold = thresholds[optimal_idx]

        # PERFORMANCE
        fpr, tpr, thresholds = sklearn.metrics.roc_curve(testXY[1], testYpred, pos_label=1)
        auc = sklearn.metrics.auc(fpr, tpr)
        
        tn, fp, fn, tp  = sklearn.metrics.confusion_matrix(testXY[1], testYpred>optimal_threshold).ravel()
        macc = (tp+tn)/(tp+tn+fp+fn)
        msen = tp/(tp+fn)
        mspc = tn/(tn+fp)
        print(f'AUC: {auc:.4f}\tAccuracy: {macc:.4f}\tSensitivity: {msen:.4f}\tSpecificity {mspc:.4f}')
    
    return model, (testXY[1], testYpred)

In [ ]:
def lstm_model(trainXY, valXY, testXY, hrs, k, arch, name, verbose=1):
    print(f'MODEL {name.upper()}', end='\t')
    
    with tf.device('/gpu:0'):
        model = Sequential()
        model.add(LSTM(input_shape=(trainXY[0].shape[1], trainXY[0].shape[2]), units=16,
                       dropout=0.0, recurrent_dropout=0.0,
                       return_sequences=True, return_state=False,
                       stateful=False, unroll=False
                      ))
        model.add(BatchNormalization())
        model.add(LSTM(units=16,
                       dropout=0.0, recurrent_dropout=0.0,
                       return_sequences=True, return_state=False,
                       stateful=False, unroll=False
                      ))
        model.add(BatchNormalization())
        model.add(LSTM(units=16,
                       dropout=0.0, recurrent_dropout=0.0,
                       return_sequences=False, return_state=False,
                       stateful=False, unroll=False
                      ))
        model.add(BatchNormalization())
        model.add(Dense(units=1, activation='sigmoid'))


    architecture = '{}-vital-{}-k{}'.format(arch, hrs, k)
    logdir = os.path.join("logs-class","{}-{}".format(architecture,
                                                      datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))

    tensorboard_callback = [tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
                            ,CustomLearningRateScheduler(lr_schedule)
                            ,EarlyStopping(monitor='val_loss', patience=10)
                            ,ModelCheckpoint(filepath="./{}/{}-best-{}-hr.h5".format(logdir,architecture,hrs)
                            ,monitor='val_loss'
                            ,save_best_only=True)]
    model.compile(loss='binary_crossentropy',
                  optimizer=tf.optimizers.Adam(0.005, beta_1=0.1, beta_2=0.001, amsgrad=True),
                  metrics=['accuracy'])
    
    with tf.device('/gpu:0'):
        model.fit(trainXY[0], trainXY[1], 
                        validation_data=valXY,
                        shuffle=True, epochs=20, verbose=verbose,
                        callbacks=[tensorboard_callback])

        validYpred = model.predict(valXY[0])
        testYpred = model.predict(testXY[0])
        
        # OPTIMAL THRESHOLD
        fpr, tpr, thresholds = sklearn.metrics.roc_curve(valXY[1], validYpred, pos_label=1)
        optimal_idx = np.argmax(tpr - fpr)
        optimal_threshold = thresholds[optimal_idx]

        # PERFORMANCE
        fpr, tpr, thresholds = sklearn.metrics.roc_curve(testXY[1], testYpred, pos_label=1)
        auc = sklearn.metrics.auc(fpr, tpr)
        
        tn, fp, fn, tp  = sklearn.metrics.confusion_matrix(testXY[1], testYpred>optimal_threshold).ravel()
        macc = (tp+tn)/(tp+tn+fp+fn)
        msen = tp/(tp+fn)
        mspc = tn/(tn+fp)
        print(f'AUC: {auc:.4f}\tAccuracy: {macc:.4f}\tSensitivity: {msen:.4f}\tSpecificity {mspc:.4f}')
    
    return model, (testXY[1], testYpred)

In [ ]:
def gru_model(trainXY, valXY, testXY, hrs, k, arch, name, verbose=1):
    print(f'MODEL {name.upper()}', end='\t')
    
    with tf.device('/gpu:0'):
        model = Sequential()
        model.add(GRU(input_shape=(trainXY[0].shape[1], trainXY[0].shape[2]), units=16,
                       dropout=0.0, recurrent_dropout=0.0,
                       return_sequences=True, return_state=False,
                       stateful=False, unroll=False
                      ))
        model.add(BatchNormalization())
        model.add(GRU(units=16,
                       dropout=0.0, recurrent_dropout=0.0,
                       return_sequences=True, return_state=False,
                       stateful=False, unroll=False
                      ))
        model.add(BatchNormalization())
        model.add(GRU(units=16,
                       dropout=0.0, recurrent_dropout=0.0,
                       return_sequences=False, return_state=False,
                       stateful=False, unroll=False
                      ))
        model.add(BatchNormalization())
        model.add(Dense(units=1, activation='sigmoid'))


    architecture = '{}-vital-{}-k{}'.format(arch, hrs, k)
    logdir = os.path.join("logs-class","{}-{}".format(architecture,
                                                      datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))

    tensorboard_callback = [tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
                            ,CustomLearningRateScheduler(lr_schedule)
                            ,EarlyStopping(monitor='val_loss', patience=10)
                            ,ModelCheckpoint(filepath="./{}/{}-best-{}-hr.h5".format(logdir,architecture,hrs)
                            ,monitor='val_loss'
                            ,save_best_only=True)]
    model.compile(loss='binary_crossentropy',
                  optimizer=tf.optimizers.Adam(0.005, beta_1=0.1, beta_2=0.001, amsgrad=True),
                  metrics=['accuracy'])
    
    with tf.device('/gpu:0'):
        model.fit(trainXY[0], trainXY[1], 
                        validation_data=valXY,
                        shuffle=True, epochs=20, verbose=verbose,
                        callbacks=[tensorboard_callback])

        validYpred = model.predict(valXY[0])
        testYpred = model.predict(testXY[0])
        
        # OPTIMAL THRESHOLD
        fpr, tpr, thresholds = sklearn.metrics.roc_curve(valXY[1], validYpred, pos_label=1)
        optimal_idx = np.argmax(tpr - fpr)
        optimal_threshold = thresholds[optimal_idx]

        # PERFORMANCE
        fpr, tpr, thresholds = sklearn.metrics.roc_curve(testXY[1], testYpred, pos_label=1)
        auc = sklearn.metrics.auc(fpr, tpr)
        
        tn, fp, fn, tp  = sklearn.metrics.confusion_matrix(testXY[1], testYpred>optimal_threshold).ravel()
        macc = (tp+tn)/(tp+tn+fp+fn)
        msen = tp/(tp+fn)
        mspc = tn/(tn+fp)
        print(f'AUC: {auc:.4f}\tAccuracy: {macc:.4f}\tSensitivity: {msen:.4f}\tSpecificity {mspc:.4f}')
    
    return model, (testXY[1], testYpred)

In [ ]:
def train_val_test(v_trainXY, v_valXY, v_testXY,
                   l_trainXY, l_valXY, l_testXY,
                   hrs, k, arch, verbose=1):

    if arch == 'DNN':
        model_vital, results_vital = baseline_model(v_trainXY, v_valXY, v_testXY, hrs, k, arch, 'vital', verbose)
        model_lab, results_lab = baseline_model(l_trainXY, l_valXY, l_testXY, hrs/8, k, arch, 'lab', verbose)
    elif arch == 'RNN':
        model_vital, results_vital = rnn_model(v_trainXY, v_valXY, v_testXY, hrs, k, arch, 'vital', verbose)
        model_lab, results_lab = rnn_model(l_trainXY, l_valXY, l_testXY, hrs/8, k, arch, 'lab', verbose)
    elif arch == 'LSTM':
        model_vital, results_vital = lstm_model(v_trainXY, v_valXY, v_testXY, hrs, k, arch, 'vital', verbose)
        model_lab, results_lab = lstm_model(l_trainXY, l_valXY, l_testXY, hrs/8, k, arch, 'lab', verbose)
    elif arch == 'GRU':
        model_vital, results_vital = gru_model(v_trainXY, v_valXY, v_testXY, hrs, k, arch, 'vital', verbose)
        model_lab, results_lab = gru_model(l_trainXY, l_valXY, l_testXY, hrs/8, k, arch, 'lab', verbose)
        
    print(f'MODEL CONCAT', end='\t')

    # Combine
    concatenated = tf.keras.layers.concatenate([model_vital.layers[-2].output, model_lab.layers[-2].output])
    if arch == 'DNN':
        out = Dense(units=64)(concatenated)
    else:
        out = Dense(units=8)(concatenated)
    out = BatchNormalization()(out)
    out = Dense(1, activation='sigmoid', name='output_layer')(out)

    model_concat = tf.keras.Model([model_vital.input, model_lab.input], out)
    if verbose == 1:
        model_concat.summary()

    architecture = '{}-vital-lab-{}'.format(arch, hrs)
    logdir = os.path.join("logs-class","{}-{}".format(architecture,
                                                      datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))

    tensorboard_callback = [tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
                            ,CustomLearningRateScheduler(lr_schedule)
                            ,EarlyStopping(monitor='val_loss', patience=5)
                            ,ModelCheckpoint(filepath="./{}/{}-best-{}-hr.h5".format(logdir,architecture,hr_data)
                            ,monitor='val_loss'
                            ,save_best_only=True)]

    model_concat.compile(loss='binary_crossentropy', 
                         optimizer=tf.optimizers.Adam(0.01, beta_1=0.1, beta_2=0.001, amsgrad=True), 
                         metrics=['accuracy'])

    with tf.device('/gpu:0'):
        model_concat.fit(x=[v_trainXY[0],l_trainXY[0]]
                        ,y=v_trainXY[1]
                        ,shuffle=True
                        ,epochs=20
                        ,verbose=verbose
                        ,validation_data=([v_valXY[0],l_valXY[0]], v_valXY[1])
                        ,callbacks=[tensorboard_callback])

        validYpred = model_concat.predict([v_valXY[0],l_valXY[0]])
        testYpred = model_concat.predict([v_testXY[0],l_testXY[0]])
        
        # OPTIMAL THRESHOLD
        fpr, tpr, thresholds = sklearn.metrics.roc_curve(v_valXY[1], validYpred, pos_label=1)
        optimal_idx = np.argmax(tpr - fpr)
        optimal_threshold = thresholds[optimal_idx]

        # PERFORMANCE
        fpr, tpr, thresholds = sklearn.metrics.roc_curve(v_testXY[1], testYpred, pos_label=1)
        auc = sklearn.metrics.auc(fpr, tpr)
        
        tn, fp, fn, tp  = sklearn.metrics.confusion_matrix(v_testXY[1], testYpred>optimal_threshold).ravel()
        macc = (tp+tn)/(tp+tn+fp+fn)
        msen = tp/(tp+fn)
        mspc = tn/(tn+fp)
        print(f'AUC: {auc:.4f}\tAccuracy: {macc:.4f}\tSensitivity: {msen:.4f}\tSpecificity {mspc:.4f}')

    results_concat = (v_testXY[1], testYpred)
    
    return model_vital, model_lab, model_concat, results_vital, results_lab, results_concat

In [ ]:
def permutation_importance(modelv, modell, modelc, datav, datal):
    pert_v = []
    pert_l = []
    pert_c = []
    print('Pertubation: Vitals')
    orig_out = modelv.predict(datav)
    for i in range(datav.shape[2]):
        new_x = datav.copy()
        perturbation = np.random.normal(0.0, 0.2, size=new_x.shape[:2])
        new_x[:, :, i] = new_x[:, :, i] + perturbation
        perturbed_out = modelv.predict(new_x)
        effect = ((orig_out - perturbed_out) ** 2).mean() ** 0.5
        print(f'Variable [{col_vital[i]: <16}], perturbation effect: {effect:.4f}')
        pert_v.append(effect)

    print('Pertubation: Labs')
    orig_out = modell.predict(datal)
    for i in range(datal.shape[2]):
        new_x = datal.copy()
        perturbation = np.random.normal(0.0, 0.2, size=new_x.shape[:2])
        new_x[:, :, i] = new_x[:, :, i] + perturbation
        perturbed_out = modell.predict(new_x)
        effect = ((orig_out - perturbed_out) ** 2).mean() ** 0.5
        print(f'Variable [{col_lab[i]: <16}], perturbation effect: {effect:.4f}')
        pert_l.append(effect)
    
    print('Pertubation: All')
    orig_out = mc.predict([datav, datal])
    for i in range(datav.shape[2]):
        new_x = datav.copy()
        perturbation = np.random.normal(0.0, 0.2, size=new_x.shape[:2])
        new_x[:, :, i] = new_x[:, :, i] + perturbation
        perturbed_out = mc.predict([new_x, datal])
        effect = ((orig_out - perturbed_out) ** 2).mean() ** 0.5
        print(f'Variable [{col_vital[i]: <16}], perturbation effect: {effect:.4f}')
        pert_c.append(effect)

    for i in range(datal.shape[2]):
        new_x = datal.copy()
        perturbation = np.random.normal(0.0, 0.2, size=new_x.shape[:2])
        new_x[:, :, i] = new_x[:, :, i] + perturbation
        perturbed_out = mc.predict([datav,new_x])
        effect = ((orig_out - perturbed_out) ** 2).mean() ** 0.5
        print(f'Variable [{col_lab[i]: <16}], perturbation effect: {effect:.4f}')
        pert_c.append(effect)
        
    pert_v = zip(col_vital, pert_v)
    pert_l = zip(col_lab, pert_l)
    pert_c = zip(col_vital + col_lab, pert_c)
    return pert_v, pert_l, pert_c

### Baseline

In [ ]:
# Baseline
num_k = 5
num_sample = db_lab.shape[0]
train_idx, test_idx = train_test_split(range(num_sample), test_size=0.15, random_state=0)

kf = KFold(n_splits=num_k, shuffle=True, random_state=0)
ktrain_idxes = []
kval_idxes = []
train_idx = np.array(train_idx)

for ktrain_idx, kval_idx in kf.split(train_idx):
    ktrain_idxes.append(train_idx[ktrain_idx])
    kval_idxes.append(train_idx[kval_idx])
    
def flat(a):
    a1 = np.reshape(a[0], (a[0].shape[0], -1))
    return (a1, a[1])

results_baseline = []
for ki in range(num_k):
    print(f'--------- k = {ki} ---------')
    VtrainXY, VvalXY, VtestXY = data_prep(db_vital, col_vital, hr_data,
                                          ktrain_idxes[ki], kval_idxes[ki], test_idx,
                                          mode='oversample', baseline=True)
    LtrainXY, LvalXY, LtestXY = data_prep(db_lab, col_lab, hr_lab,
                                          ktrain_idxes[ki], kval_idxes[ki], test_idx,
                                          mode='oversample', baseline=True)
    mv, ml, mc, rv, rl, rc = train_val_test(flat(VtrainXY),flat(VvalXY),
                                            flat(VtestXY),flat(LtrainXY),
                                            flat(LvalXY),flat(LtestXY)
                                            ,48,0,'DNN',0)
    results_baseline.append(rc)

In [ ]:
pd.DataFrame({'data': [results_baseline]}).to_pickle('results_baseline{}.pickle'.format(hr_data))

### RNN

In [ ]:
#RNN
num_k = 5
num_sample = db_lab.shape[0]
train_idx, test_idx = train_test_split(range(num_sample), test_size=0.15, random_state=0)

kf = KFold(n_splits=num_k, shuffle=True, random_state=0)
train_idx = np.array(train_idx)
ktrain_idxes = []
kval_idxes = []
for ktrain_idx, kval_idx in kf.split(train_idx):
    ktrain_idxes.append(train_idx[ktrain_idx])
    kval_idxes.append(train_idx[kval_idx])
    
results_rnn = []
for ki in range(num_k):
    print(f'--------- k = {ki} ---------')
    VtrainXY, VvalXY, VtestXY = data_prep(db_vital, col_vital, hr_data,
                                          ktrain_idxes[ki], kval_idxes[ki], test_idx,
                                          mode='oversample', baseline=False)
    LtrainXY, LvalXY, LtestXY = data_prep(db_lab, col_lab, hr_lab,
                                          ktrain_idxes[ki], kval_idxes[ki], test_idx,
                                          mode='oversample', baseline=False)
    mv, ml, mc, rv, rl, rc = train_val_test(VtrainXY,VvalXY,VtestXY,LtrainXY,LvalXY,LtestXY,48,0,'RNN',0)
    results_rnn.append(rc)

In [ ]:
pd.DataFrame({'data': [results_rnn]}).to_pickle('results_rnn{}.pickle'.format(hr_data))

### LSTM

In [ ]:
# LSTM
num_k = 5
num_sample = db_lab.shape[0]
train_idx, test_idx = train_test_split(range(num_sample), test_size=0.15, random_state=0)

kf = KFold(n_splits=num_k, shuffle=True, random_state=0)
ktrain_idxes = []
kval_idxes = []
train_idx = np.array(train_idx)

for ktrain_idx, kval_idx in kf.split(train_idx):
    ktrain_idxes.append(train_idx[ktrain_idx])
    kval_idxes.append(train_idx[kval_idx])
    
results_lstm = []
for ki in range(num_k):
    print(f'--------- k = {ki} ---------')
    VtrainXY, VvalXY, VtestXY = data_prep(db_vital, col_vital, hr_data,
                                          ktrain_idxes[ki], kval_idxes[ki], test_idx,
                                          mode='oversample', baseline=False)
    LtrainXY, LvalXY, LtestXY = data_prep(db_lab, col_lab, hr_lab,
                                          ktrain_idxes[ki], kval_idxes[ki], test_idx,
                                          mode='oversample', baseline=False)
    mv, ml, mc, rv, rl, rc = train_val_test(VtrainXY,VvalXY,VtestXY,LtrainXY,LvalXY,LtestXY,48,0,'LSTM',0)
    results_lstm.append(rc)

In [ ]:
pd.DataFrame({'data': [results_lstm]}).to_pickle('results_lstm{}.pickle'.format(hr_data))

### GRU

In [ ]:
# GRU
num_k = 5
num_sample = db_lab.shape[0]
train_idx, test_idx = train_test_split(range(num_sample), test_size=0.15, random_state=0)

kf = KFold(n_splits=num_k, shuffle=True, random_state=0)
ktrain_idxes = []
kval_idxes = []
train_idx = np.array(train_idx)

for ktrain_idx, kval_idx in kf.split(train_idx):
    ktrain_idxes.append(train_idx[ktrain_idx])
    kval_idxes.append(train_idx[kval_idx])
    
results_gru = []
for ki in range(num_k):
    print(f'--------- k = {ki} ---------')
    VtrainXY, VvalXY, VtestXY = data_prep(db_vital, col_vital, hr_data,
                                          ktrain_idxes[ki], kval_idxes[ki], test_idx,
                                          mode='oversample', baseline=False)
    LtrainXY, LvalXY, LtestXY = data_prep(db_lab, col_lab, hr_lab,
                                          ktrain_idxes[ki], kval_idxes[ki], test_idx,
                                          mode='oversample', baseline=False)
    mv, ml, mc, rv, rl, rc = train_val_test(VtrainXY,VvalXY,VtestXY,LtrainXY,LvalXY,LtestXY,48,0,'GRU',0)
    results_gru.append(rc)

In [ ]:
pd.DataFrame({'data': [results_gru]}).to_pickle('results_gru{}.pickle'.format(hr_data))

In [ ]:
d ={'m3-baseline': [results_baseline],
    'm3-rnn': [results_rnn],
    'm3-lstm': [results_lstm],
    'm3-gru': [results_gru]
   }

df= pd.DataFrame(data=d)
df.to_pickle('result-m3.pickle')

### Feature Importance with SHAP

In [ ]:
import shap

In [ ]:
shv = []
for i in range(5):
    mv = tf.keras.models.load_model(f'mv-{i}')
    explainer = shap.DeepExplainer(mv, VtrainXY[0])
    shap_values = explainer.shap_values(VtestXY[0])
    shv.append(shap_values)

shl = []
for i in range(5):
    ml = tf.keras.models.load_model(f'ml-{i}')
    explainer = shap.DeepExplainer(ml, LtrainXY[0])
    shap_values = explainer.shap_values(LtestXY[0])
    shl.append(shap_values)

In [ ]:
np.array(shv).save('m3-shv')
np.array(shl).save('m3-shl')